In [1]:
from scipy.stats import skew,kurtosis,iqr
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import numpy as np
sys.path.append('../../LIB/')
from env import ENV
import pickle
import warnings

from functools import partial
from sklearn.externals import joblib
import warnings
warnings.filterwarnings('ignore')
import multiprocessing as mp
from functools import reduce

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
def calculate_na(ser):
    return np.sum(ser.isnull())


def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

In [3]:
train = pd.read_pickle(ENV.application_train_reproduce.value)
test = pd.read_pickle(ENV.application_test_reproduce.value)

In [4]:
p = scan_nan_portion(train)
ignore = list(p[p>0.95].index.values)

In [5]:
train = train[list(set(train.columns.values) - set(ignore))]
test = test[list(set(test.columns.values) - set(ignore))]

In [6]:
p = scan_nan_portion(test)
p.describe()

count    1087.000000
mean        0.135475
std         0.226418
min         0.000000
25%         0.016412
50%         0.019202
75%         0.131791
max         0.781963
dtype: float64

In [9]:
for col in train.columns:
    if train[col].dtype == object:
        print(col)
        le.fit(pd.concat([train[col],test[col]]).fillna('NA'))
        train[col] = le.transform(train[col].fillna('NA'))
        test[col] = le.transform(test[col].fillna('NA'))
#         le.fit(pd.concat([train[col],test[col]]))
#         train[col] = le.transform(train[col])
#         test[col] = le.transform(test[col])

In [12]:
train.TARGET.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [13]:
train.to_pickle('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project_home_credit/data/fe_application_train.pkl')
test.to_pickle('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project_home_credit/data/fe_application_test.pkl')

In [5]:
# train.to_pickle(ENV.application_train_reproduce.value)
# test.to_pickle(ENV.application_test_reproduce.value)

In [14]:
train.dtypes.value_counts()

float64    1030
int64        58
dtype: int64

In [7]:
test.shape

(48744, 1318)